### Basic ML Model Monitoring with MLFlow
**Description**: Set up a basic ML model monitoring pipeline using MLFlow to track experiment parameters and results.

**Steps**:
1. Installation
2. Set Up MLFlow Tracking
3. Training a Simple Model
4. Logging Model and Metrics
5. View Logged Data
    - Access the MLFlow UI to view your logged parameters, metrics, and models.

In [1]:
# write your code from here

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.sklearn
import os

# -------------------------------
# Step 1: Simulate Sample Data
# -------------------------------
def generate_user_data():
    """
    Simulate binary classification dataset for ML monitoring.
    """
    np.random.seed(42)
    size = 1000
    df = pd.DataFrame({
        "session_length": np.random.normal(300, 50, size),
        "click_rate": np.random.beta(2, 5, size),
        "prior_purchases": np.random.poisson(2, size),
        "converted": np.random.choice([0, 1], size=size, p=[0.7, 0.3])
    })
    return df


# -------------------------------
# Step 2: Train a Simple Model
# -------------------------------
def train_model(df):
    """
    Train logistic regression and return model and metrics.
    """
    X = df.drop("converted", axis=1)
    y = df["converted"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = LogisticRegression()
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred),
        "confusion_matrix": confusion_matrix(y_test, y_pred).tolist()
    }

    return model, scaler, metrics


# -------------------------------
# Step 3: Log to MLFlow
# -------------------------------
def log_with_mlflow(model, scaler, metrics):
    """
    Logs parameters, metrics, and artifacts to MLflow.
    """
    mlflow.set_experiment("Basic_ML_Monitoring")

    with mlflow.start_run():
        mlflow.set_tag("project", "model_monitoring")
        mlflow.set_tag("author", "student")

        # Log parameters
        mlflow.log_param("model_type", "LogisticRegression")
        mlflow.log_param("scaling", "StandardScaler")

        # Log metrics
        for key, value in metrics.items():
            if isinstance(value, (int, float)):
                mlflow.log_metric(key, value)

        # Log models
        mlflow.sklearn.log_model(model, artifact_path="logistic_model")
        mlflow.sklearn.log_model(scaler, artifact_path="scaler")

        print(f"[MLflow] Experiment logged with accuracy = {metrics['accuracy']:.4f}")


# -------------------------------
# Step 4: Main Pipeline
# -------------------------------
def main():
    print("=== MLflow: Basic ML Model Monitoring ===")

    # Step 1: Data
    data = generate_user_data()

    # Step 2: Training
    model, scaler, metrics = train_model(data)

    # Step 3: Logging
    log_with_mlflow(model, scaler, metrics)

    print("\n✅ Monitoring complete. Run 'mlflow ui' to view the experiment.")

# -------------------------------
# Run Everything
# -------------------------------
if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'mlflow'